In [ ]:
# | default_exp blocks/cnn

# Imports

In [ ]:
# | export


from typing import Any, Literal

import torch
from torch import nn

from vision_architectures.utils.activation_checkpointing import ActivationCheckpointing
from vision_architectures.utils.activations import get_act_layer
from vision_architectures.utils.custom_base_model import CustomBaseModel
from vision_architectures.utils.normalizations import get_norm_layer
from vision_architectures.utils.rearrange import rearrange_channels

# Config

In [ ]:
# | export


class CNNBlock3DConfig(CustomBaseModel):
    in_channels: int
    out_channels: int
    kernel_size: int
    padding: int | tuple[int, ...] | str = "same"
    stride: int = 1
    conv_kwargs: dict[str, Any] = {}

    sequence: Literal["ADN", "AND", "DAN", "DNA", "NAD", "NDA"] = "NDA"

    normalization: str | None = None
    drop_prob: float = 0.0
    activation: str | None = None

# Architecture

In [ ]:
# | export


class CNNBlock3D(nn.Module):
    def __init__(self, config: CNNBlock3DConfig = {}, checkpointing_level: int = 0, **kwargs):
        super().__init__()

        self.config = CNNBlock3DConfig.model_validate(config | kwargs)

        normalization = self.config.normalization
        activation = self.config.activation
        drop_prob = self.config.drop_prob
        sequence = self.config.sequence

        bias = True
        if normalization is not None and normalization.startswith("batchnorm") and sequence.startswith("N"):
            bias = False
        self.cnn = nn.Conv3d(
            in_channels=self.config.in_channels,
            out_channels=self.config.out_channels,
            kernel_size=self.config.kernel_size,
            padding=self.config.padding,
            stride=self.config.stride,
            bias=bias,
            **self.config.conv_kwargs,
        )

        self.norm_layer = get_norm_layer(normalization, self.config.out_channels)
        self.act_layer = get_act_layer(activation)
        self.dropout = nn.Dropout(drop_prob)

        self.checkpointing_level1 = ActivationCheckpointing(1, checkpointing_level)

    def _forward(self, x: torch.Tensor, channels_first: bool = True):
        # x: (b, [in_channels], z, y, x, [in_channels])

        x = rearrange_channels(x, channels_first, True)
        # Now x is (b, in_channels, z, y, x)

        x = self.cnn(x)
        # Now x is (b, out_channels, z, y, x)

        for layer in self.config.sequence:
            if layer == "A":
                x = self.act_layer(x)
            elif layer == "D":
                x = self.dropout(x)
            elif layer == "N":
                x = self.norm_layer(x)
            # (b, out_channels, z, y, x)

        x = rearrange_channels(x, True, channels_first)
        # (b, [out_channels], z, y, x, [out_channels])

        return x

    def forward(self, *args, **kwargs):
        return self.checkpointing_level1(self._forward, *args, **kwargs)

In [ ]:
test = CNNBlock3D(
    in_channels=4,
    out_channels=8,
    kernel_size=3,
    normalization="batchnorm3d",
    activation="silu",
    drop_prob=0.5,
    padding=1,
    conv_kwargs={"groups": 2},
    sequence="NDA",
)
display(test)

sample_input = torch.randn(2, 4, 16, 16, 16)
test(sample_input).shape


CNNBlock3D(
  (cnn): Conv3d(4, 8, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), groups=2, bias=False)
  (norm_layer): BatchNorm3d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act_layer): SiLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (checkpointing_level1): ActivationCheckpointing(enabled=False, checkpointing_level=1)
)

torch.Size([2, 8, 16, 16, 16])

# nbdev

In [ ]:
!nbdev_export